# mimic LOB - DEMO


# 0. Imports

In [8]:
# Imports
from   IPython.display import display, HTML
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import os
from pathlib import Path

# parent directory
dirname=os.path.dirname

# Display options
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

In [9]:
import mimicLOB as mlob

# package's path
lobpath = dirname(os.path.abspath(mlob.__file__)).replace('\\', '/') + '/LOBserver.py'
fixpath = dirname(os.path.abspath(mlob.__file__)).replace('\\', '/') + '/FIXserver.py'

# local server
localserver = "http://127.0.0.1:5000"
distantserver = "http://fdr0903.pythonanywhere.com/"

# Server used
server = localserver

#### launch the distant LOB server, if you choose the localserver option
#### Copy the command in a terminal

In [3]:
port = 5000
f"python {lobpath} {port}"

'python C:/Users/fayca/OneDrive/Documents/FRDev/mimicLOB/mimicLOB/LOBserver.py 5000'

#### Open the 5003 port in your local host to receive messages from the distant LOB

In [4]:
port = 5003
f"python {fixpath} {port}"

'python C:/Users/fayca/OneDrive/Documents/FRDev/mimicLOB/mimicLOB/FIXserver.py 5003'

# 2. Agents
The agents is constantly connected to the distant LOB via the server address.

#### Parameters :
1. distant : if true, the agent must have the server address. If False, he should have the orderbook object passed in the configuration.
2. server : when distant is True, the http address of the server the LOB is running on.
3. id : optional, default is generic. the if is used in the transaction tape.
4. b_record : optional, boolean, default is False. If True, the agent records all his activity : sent orders, executed trades, pending orders ...
5. FIXaddress : address & port on which the agent is listening to the LOB messages. Every agent should have a different port.


In [5]:
EURONEXT_config = {'distant' : True,
                   'server'  : server,
                   'id'      : 'market'}

myself_config = {'distant' : True,
                 'server'  : server,
                 'id'      : 'FDR',
                 'b_record': True,
                 'FIXaddress' : 'http://127.0.0.1:5003/',
                 'usengrok'   : False}

other_config = {'distant'  : True,
                'server'   : server,
                'id'       : 'Olivier'}

euronext = mlob.genericAgent(**EURONEXT_config)
myself   = mlob.genericAgent(**myself_config)
other    = mlob.genericAgent(**other_config)

#### Fix server : local host is accessible via this public adderess. IT is used by the distant LOB to send notifications of trades & orders sitting in the book.

In [6]:
myself._fixserver

'http://789076d0.ngrok.io'

#### The agent with id 'market' can change the LOB configuration and reset it

In [7]:
euronext.setLOB_tickSize(0.1)

'DONE'

# 3. Get the LOB state
Either agents have access to LOB information

In [ ]:
LOBstate = euronext.getLOBState()
tickSize =  float(euronext.getTickSize())
print(f'tick size : {tickSize}')
if type(LOBstate) == pd.DataFrame:
    if len(LOBstate) >0 :
        LOBstate = LOBstate.set_index('Price').sort_index()
        minPrice = LOBstate.index[0]
        maxPrice = LOBstate.index[-1]
        LOBstate = LOBstate.reindex(np.arange(minPrice, maxPrice+tickSize, tickSize))
        LOBstate.plot.bar(figsize=(20, 7))
    else:
        print('LOB is empty')
else:
    display(LOBstate)

# 4. Send orders

#### limit sell order @101 & limit buy order @99
Check the LOB state afterwards

In [ ]:
# Send Orders
myself.send_sell_limit_order(quantity = 100,
                             price    = 101)

myself.send_buy_limit_order(quantity = 100,
                             price    = 100)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

#### match orders
Olivier will sell 50 at 100. A transaction will happen.

In [ ]:
other.send_sell_limit_order(quantity = 50,
                             price    = 100)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

# 5. Modify orders

Rules for modification : 
1. If the order is already executed : no new order
2. Any order entered into the Central Order Book may be modified or cancelled prior to its execution. Any increase in the order quantity or change in the limit price shall cause the forfeiture of time priority. (Rule 4202/4 of Euronext)

Rules for cancellation :
1. If the order is already executed : no order cancellation

#### Cancel an order

In [ ]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

In [ ]:
myself.cancelOrder(side     = 'ask', 
                   order_id = 1)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

#### modify an order

In [ ]:
myself.modifyOrder(order_id     = 2, 
                   side         = 'bid',
                   new_price    = 5,
                   new_quantity = 56)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

# 6. Modify The LOB
One agent type (market) can reset the LOB, dump the tape, and modify the LOB properties 
#### Change tick size

In [ ]:
euronext.setLOB_tickSize(0.1)

#### Remove all pending orders

In [ ]:
euronext.resetLOB_PendingOrders()

#### Remove orders and transactions

In [ ]:
euronext.resetLOB()